In [ ]:
import pydicom
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def load_dicom_series(directory):
    slices = []
    for filename in sorted(os.listdir(directory)):
        if filename.endswith('.dcm'):
            filepath = os.path.join(directory, filename)
            ds = pydicom.dcmread(filepath)
            slices.append(ds)
    slices.sort(key=lambda x: int(x.InstanceNumber))
    return slices

def apply_gamma_correction(image, gamma=1.0):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")

    return cv2.LUT(image, table)

def preprocess_slice(slice, hu_min=-900, hu_max=400):
    image = slice.pixel_array.astype(np.float32)

    intercept = slice.RescaleIntercept if 'RescaleIntercept' in slice else 0
    slope = slice.RescaleSlope if 'RescaleSlope' in slice else 1
    image = image * slope + intercept

    image = np.clip(image, hu_min, hu_max)
    image = (image - hu_min) / (hu_max - hu_min) * 255

    original = image.astype(np.uint8).copy()

    image = apply_gamma_correction(image.astype(np.uint8), 10)

    sub = cv2.subtract(image, original)
    image = cv2.subtract(original, sub)

    return image

def plot_histogram(image, hu_min=-1000, hu_max=400):
    plt.hist(image.ravel(), bins=256, range=[hu_min, hu_max])
    plt.xlabel('Hounsfield Units (HU)')
    plt.ylabel('Frequency')
    plt.title('Histogram of CT Image Intensities')
    plt.show()

def segment_ROI(image, threshold_min=10, threshold_max=70):
    binary_image = cv2.inRange(image, threshold_min, threshold_max)
    binary_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))

    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        cv2.drawContours(binary_image, [cnt], 0, 255, -1)

    return binary_image

def postprocess_mask(mask):
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))
    mask = cv2.erode(mask, np.ones((5, 5), np.uint8), iterations=5)
    return mask

def segment_lungs_from_initial_mask(original, preprocessed, initial_mask):
    masked_image = cv2.bitwise_and(original, original, mask=initial_mask)

    threshold_min = -900
    threshold_max = 550
    lung_mask = cv2.inRange(masked_image, threshold_min, threshold_max)
    lung_mask = cv2.bitwise_and(lung_mask, lung_mask, mask=initial_mask)

    # Encontrar contornos
    contours, _ = cv2.findContours(lung_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Ordenar contornos por área (do maior para o menor)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    # Criar uma nova máscara apenas com os dois maiores contornos (pulmões)
    new_mask = np.zeros(lung_mask.shape, dtype=np.uint8)
    cv2.drawContours(new_mask, contours[:2], -1, 255, -1)
    
    # Aplicar a nova máscara
    lung_mask = cv2.bitwise_and(lung_mask, new_mask)

    # Aplicar operações morfológicas
    kernel = np.ones((5,5), np.uint8)
    lung_mask = cv2.morphologyEx(lung_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    lung_mask = cv2.morphologyEx(lung_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    return masked_image, lung_mask



def segment_nodules(lung_mask, original):
    # Aplicar a máscara dos pulmões à imagem original
    masked_image = cv2.bitwise_and(original, original, mask=lung_mask)
    # masked_image = cv2.medianBlur(masked_image, 5)
    # masked_image = cv2.equalizeHist(masked_image)

    # Aplicar thresholding para segmentar nódulos
    threshold_min = 500
    threshold_max = 900
    nodule_mask = cv2.inRange(masked_image, threshold_min, threshold_max)
    # Pós-processamento para refinar a segmentação dos nódulos
    nodule_mask = cv2.morphologyEx(nodule_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    nodule_mask = cv2.morphologyEx(nodule_mask, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8))

    nodule_mask = cv2.bitwise_and(nodule_mask, nodule_mask, mask=lung_mask)



    return nodule_mask

def visualize_segmentation(original, preprocessed, initial_mask, masked_image, lung_mask, nodule_mask):
    if original.dtype != np.uint8:
        original = cv2.normalize(original, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    overlay_initial = np.zeros((original.shape[0], original.shape[1], 3), dtype=np.uint8)
    overlay_initial[initial_mask == 255] = [255, 0, 0]  # vermelho

    combined_initial = cv2.addWeighted(cv2.cvtColor(original, cv2.COLOR_GRAY2BGR), 0.7, overlay_initial, 0.3, 0)

    overlay_lung = np.zeros((original.shape[0], original.shape[1], 3), dtype=np.uint8)
    overlay_lung[lung_mask == 255] = [0, 255, 0]  # verde

    combined_lung = cv2.addWeighted(cv2.cvtColor(original, cv2.COLOR_GRAY2BGR), 0.7, overlay_lung, 0.3, 0)

    overlay_nodule = np.zeros((original.shape[0], original.shape[1], 3), dtype=np.uint8)
    overlay_nodule[nodule_mask == 255] = [0, 0, 255]  # azul

    combined_nodule = cv2.addWeighted(cv2.cvtColor(original, cv2.COLOR_GRAY2BGR), 0.7, overlay_nodule, 0.3, 0)

    plt.figure(figsize=(30, 5))

    plt.subplot(1, 6, 1)
    plt.title('Original Slice')
    plt.imshow(original, cmap='gray')

    plt.subplot(1, 6, 2)
    plt.title('Preprocessed Slice')
    plt.imshow(preprocessed, cmap='gray')

    plt.subplot(1, 6, 3)
    plt.title('Initial Mask')
    plt.imshow(combined_initial)

    plt.subplot(1, 6, 4)
    plt.title('Masked Image')
    plt.imshow(masked_image, cmap='gray')

    plt.subplot(1, 6, 5)
    plt.title('Lung Mask')
    plt.imshow(combined_lung)

    plt.subplot(1, 6, 6)
    plt.title('Nodule Mask')
    plt.imshow(combined_nodule)

    plt.show()

directory = 'C:/Users/nicol/OneDrive/Documentos/PDI-Final-Article/dataset/exam01/data'
slices = load_dicom_series(directory)

original_slices = [slice.pixel_array for slice in slices]

preprocessed_slices = [preprocess_slice(slice) for slice in slices]

# Plotar histogramas para cada slice (opcional)
# for slice in preprocessed_slices:
#     plot_histogram(slice)

# Segmentação inicial
threshold_min = 30
threshold_max = 255
initial_masks = [segment_ROI(slice, threshold_min, threshold_max) for slice in preprocessed_slices]

# Pós-processar as máscaras iniciais
postprocessed_initial_masks = [postprocess_mask(mask) for mask in initial_masks]

# Segmentar os pulmões a partir da máscara inicial
segmentation_results = [segment_lungs_from_initial_mask(original, preprocessed, initial_mask) for original, preprocessed, initial_mask in zip(original_slices, preprocessed_slices, postprocessed_initial_masks)]
masked_images, lung_masks = zip(*segmentation_results)

# Segmentar nódulos pulmonares
nodule_masks = [segment_nodules(lung_mask, original) for lung_mask, original in zip(lung_masks, original_slices)]

# Visualizar os resultados a partir do slice 38 (índice 37)
for i in range(37, 47):
    original = original_slices[i]
    preprocessed = preprocessed_slices[i]
    initial_mask = postprocessed_initial_masks[i]
    masked_image = masked_images[i]
    lung_mask = lung_masks[i]
    nodule_mask = nodule_masks[i]
    visualize_segmentation(original, preprocessed, initial_mask, masked_image, lung_mask, nodule_mask)


In [ ]:
import nibabel as nib

def load_ground_truth_masks(exam_directory):
    lung_mask_path = os.path.join(exam_directory, 'mask', 'Untitled.nii.gz')
    nodule_mask_path = os.path.join(exam_directory, 'nodules', 'nodule_exam01_masks.nii.gz')
    
    lung_gt_img = nib.load(lung_mask_path)
    nodule_gt_img = nib.load(nodule_mask_path)
    
    lung_gt_masks = lung_gt_img.get_fdata()
    nodule_gt_masks = nodule_gt_img.get_fdata()
    
    return lung_gt_masks, nodule_gt_masks

In [ ]:
import numpy as np

def dice_coefficient(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    return 2. * intersection.sum() / (y_true.sum() + y_pred.sum())

def jaccard_index(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    return intersection.sum() / union.sum()

def sensitivity(y_true, y_pred):
    true_positives = np.logical_and(y_true, y_pred).sum()
    false_negatives = np.logical_and(y_true, np.logical_not(y_pred)).sum()
    return true_positives / (true_positives + false_negatives)

def specificity(y_true, y_pred):
    true_negatives = np.logical_and(np.logical_not(y_true), np.logical_not(y_pred)).sum()
    false_positives = np.logical_and(np.logical_not(y_true), y_pred).sum()
    return true_negatives / (true_negatives + false_positives)

def accuracy(y_true, y_pred):
    correct = np.equal(y_true, y_pred).sum()
    total = y_true.size
    return correct / total

def calculate_metrics(y_true, y_pred):
    return {
        'Dice Coefficient': dice_coefficient(y_true, y_pred),
        'Jaccard Index': jaccard_index(y_true, y_pred),
        'Sensitivity': sensitivity(y_true, y_pred),
        'Specificity': specificity(y_true, y_pred),
        'Accuracy': accuracy(y_true, y_pred)
    }


In [ ]:
import os
import csv
import numpy as np
import nibabel as nib

def load_ground_truth_masks(exam_directory):
    lung_mask_path = os.path.join(exam_directory, 'mask', 'Untitled.nii.gz')
    nodule_mask_path = os.path.join(exam_directory, 'nodules', 'nodule_exam01_masks.nii.gz')
    
    lung_gt_img = nib.load(lung_mask_path)
    nodule_gt_img = nib.load(nodule_mask_path)
    
    lung_gt_masks = lung_gt_img.get_fdata()
    nodule_gt_masks = nodule_gt_img.get_fdata()
    
    return lung_gt_masks, nodule_gt_masks

def evaluate_segmentation(pred_mask, gt_mask):
    # Certifique-se de que as máscaras são binárias
    pred_mask = (pred_mask > 0).astype(np.uint8)
    gt_mask = (gt_mask > 0).astype(np.uint8)
    
    metrics = calculate_metrics(gt_mask, pred_mask)
    return metrics

def save_metrics_to_csv(lung_metrics, nodule_metrics, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Slice', 'Metric', 'Lung', 'Nodule'])
        for i in range(len(lung_metrics)):
            for metric in lung_metrics[i].keys():
                writer.writerow([i, metric, lung_metrics[i][metric], nodule_metrics[i][metric]])

# Diretório principal dos exames
base_directory = 'C:/Users/nicol/OneDrive/Documentos/PDI-Final-Article/dataset'

# Nome do exame a ser processado
exam_name = 'exam01'

# Diretório do exame específico
exam_directory = os.path.join(base_directory, exam_name)

if os.path.isdir(exam_directory):
    data_directory = os.path.join(exam_directory, 'data')
    slices = load_dicom_series(data_directory)
    
    original_slices = [slice.pixel_array for slice in slices]
    preprocessed_slices = [preprocess_slice(slice) for slice in slices]
    
    # Carregar as máscaras ground truth
    lung_gt_masks, nodule_gt_masks = load_ground_truth_masks(exam_directory)
    
    # Segmentação inicial
    threshold_min = 30
    threshold_max = 255
    initial_masks = [segment_ROI(slice, threshold_min, threshold_max) for slice in preprocessed_slices]
    
    # Pós-processar as máscaras iniciais
    postprocessed_initial_masks = [postprocess_mask(mask) for mask in initial_masks]
    
    # Segmentar os pulmões a partir da máscara inicial
    segmentation_results = [segment_lungs_from_initial_mask(original, preprocessed, initial_mask) for original, preprocessed, initial_mask in zip(original_slices, preprocessed_slices, postprocessed_initial_masks)]
    masked_images, lung_masks = zip(*segmentation_results)
    
    # Segmentar nódulos pulmonares
    nodule_masks = [segment_nodules(lung_mask, original) for lung_mask, original in zip(lung_masks, preprocessed_slices)]
    
    # Listas para armazenar as métricas de cada slice
    lung_metrics_exam = []
    nodule_metrics_exam = []
    
    for i in range(5):
        lung_mask = lung_masks[i]
        nodule_mask = nodule_masks[i]
        lung_gt = lung_gt_masks[:, :, i]
        nodule_gt = nodule_gt_masks[:, :, i]
        
        # Calcular métricas
        lung_metrics = evaluate_segmentation(lung_mask, lung_gt)
        nodule_metrics = evaluate_segmentation(nodule_mask, nodule_gt)
        
        lung_metrics_exam.append(lung_metrics)
        nodule_metrics_exam.append(nodule_metrics)
        
        print(f"Slice {i}:")
        print("Lung Segmentation Metrics:")
        for metric, value in lung_metrics.items():
            print(f"{metric}: {value:.4f}")
        print("\nNodule Segmentation Metrics:")
        for metric, value in nodule_metrics.items():
            print(f"{metric}: {value:.4f}")
        print("\n")
        
        # Visualizar a segmentação (opcional)
        visualize_segmentation(original_slices[i], preprocessed_slices[i], 
                               postprocessed_initial_masks[i], masked_images[i], 
                               lung_mask, nodule_mask)
    
    # Salvar as métricas em um arquivo CSV
    save_metrics_to_csv(lung_metrics_exam, nodule_metrics_exam, f'{exam_name}_segmentation_metrics.csv')
